In [20]:
from operator import attrgetter
from math import sqrt
import random

In [24]:
class Node:
    def __init__(self, _id, x, y, z, adjacent=None):
        self._id = _id
        self.x = x
        self.y = y
        self.z = z
        self.adjacent = adjacent or list()
        
    def __repr__(self):
        string = "{}: (x: {}, y: {}, z: {}) -> {}"
        details = attrgetter('_id', 'x', 'y', 'z')(self)
        return string.format(*details, [n._id for n in self.adjacent])
    
    def __str__(self):
        return self.__repr__()
    
    def connect_to(self, node):
        self.adjacent.append(node)
        node.adjacent.append(self)
        
    @staticmethod
    def random_coordinate_generator():
        x = random.randrange(10, 10000)/100
        y = random.randrange(10, 10000)/100
        z = random.randrange(10, 500)/100
        return (x, y, z)
    
    @staticmethod
    def distance_between(n1, n2):
        distance = sqrt((n1.x-n2.x)**2 + (n1.y-n2.y)**2 + (n1.z-n2.z)**2)
        return distance
    
    def distance_from(self, other):
        return Node.distance_between(self, other)
    
class Graph:
    def __init__(self, nodes=None, max_connections=3):
        self.nodes = nodes or list()
        self.max_connections = max_connections
        
    def __repr__(self):
        return '\n'.join(map(str, self.nodes))
    
    def __str__(self):
        nodes = []
        edges = []
        
        for node in self.nodes:
            nodes.append((node.x, node.y, node.z))
            for adj in node.adjacent:
                edges.append((node._id, adj._id))
                
        nodes = '\n'.join((','.join(map(str, coord)) for coord in nodes))
        edges = '\n'.join((','.join(map(str, _ids)) for _ids in edges))
        
        return nodes + '\n\n' + edges
    
    def add_node(self, node=None):
        if node is None:
            _id = max(n._id for n in self.nodes)+1 if self.nodes else 0
            node = Node(_id, *Node.random_coordinate_generator())
            self.nodes.append(node)
        else:
            self.nodes.append(nodes)
            
    def connect(self, n1, n2, x_dist=45, y_dist=45, z_dist=15):
        if n1 in self.nodes and n2 in self.nodes:
            computed_x_dist = abs(n1.x - n2.x)
            computed_y_dist = abs(n1.y - n2.y)
            computed_z_dist = abs(n1.z - n2.z)
            if computed_x_dist < x_dist and computed_y_dist < y_dist and computed_z_dist < z_dist:
                if len(n1.adjacent) <= self.max_connections and len(n2.adjacent) <= self.max_connections:
                    n1.connect_to(n2)
            
    def seed(self, n=10):
        # add nodes
        for _ in range(n):
            self.add_node()
            
        # create_edges
        for node in self.nodes:
            adjacent = random.sample(self.nodes, random.randrange(len(self.nodes)))
            for n in adjacent:
                self.connect(node, n)
            

graph = Graph()
graph.seed(20)

graph


0: (x: 41.54, y: 46.26, z: 4.86) -> [1, 3, 15, 16]
1: (x: 83.37, y: 18.74, z: 1.21) -> [0, 1, 1, 2]
2: (x: 38.4, y: 46.28, z: 3.19) -> [1, 4, 10, 14]
3: (x: 5.32, y: 28.19, z: 0.33) -> [16, 0, 3, 3]
4: (x: 72.04, y: 14.42, z: 0.55) -> [2, 13, 17]
5: (x: 27.92, y: 93.59, z: 1.64) -> [11, 14, 19, 15]
6: (x: 52.31, y: 10.15, z: 1.65) -> [13, 6, 6, 15]
7: (x: 40.29, y: 39.77, z: 4.32) -> [18, 7, 7, 19]
8: (x: 86.88, y: 99.38, z: 1.32) -> [12, 12, 14]
9: (x: 19.48, y: 93.97, z: 3.34) -> [12]
10: (x: 66.43, y: 84.24, z: 3.89) -> [2, 19, 10, 10]
11: (x: 39.46, y: 96.59, z: 2.65) -> [5, 19]
12: (x: 59.41, y: 65.61, z: 4.7) -> [8, 14, 8, 9]
13: (x: 85.23, y: 6.46, z: 4.34) -> [6, 17, 4]
14: (x: 51.12, y: 74.0, z: 1.7) -> [2, 5, 12, 8]
15: (x: 69.51, y: 53.87, z: 3.43) -> [5, 6, 0, 17]
16: (x: 1.91, y: 3.75, z: 2.08) -> [3, 0, 16, 16]
17: (x: 41.55, y: 28.73, z: 2.03) -> [13, 15, 4, 18]
18: (x: 8.16, y: 45.51, z: 2.45) -> [7, 17, 18, 18]
19: (x: 67.83, y: 63.0, z: 1.38) -> [5, 7, 10, 11]

In [25]:
from splinter import Browser

browser = Browser(driver_name='phantomjs')
browser.visit('https://geo.improbable.io/solutions-networkviewer/')
browser.find_by_id('dataArea').fill(str(graph))
browser.find_by_id('process').click()
svg = browser.find_by_id('view').html.strip()

In [26]:
from IPython.display import display, SVG
display(SVG(svg))